# Make your own wildfires dataset:

You need to have [google earth engine ](https://earthengine.google.com/) account


In [ ]:
!pip install earthengine-api

In [ ]:
import ee
import datetime
import pandas as pd
import numpy as np

ee.Authenticate()
ee.Initialize()

this function create a dataset based on the [MOD14A1.061: Terra Thermal Anomalies & Fire Daily Global 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD14A1) product

In [150]:
def wildfire_dataset(province, town, coordinates,	start_date, end_date):
  N = 10 ### number of rows if there is data (N x 5000)
  aoi = ee.Geometry.Polygon(list(coordinates))
  modis_collection = ee.ImageCollection('MODIS/061/MOD14A1')
  filtered_collection = modis_collection.filterBounds(aoi).filterDate(start_date, end_date)

  def add_date(feature, date):
    return feature.set('date', date)
  def sample_image(image, band):
      date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
      return image.select(band).sample(region=aoi,  geometries = True).map(lambda feature: add_date(feature, date))

  def make_dataset(band):

    feature_collection = filtered_collection.map(lambda image: sample_image(image, band))
    data_sampled = feature_collection.flatten()
    features_lst = []
    for i in range(N):
      features_lst = features_lst + data_sampled.toList(5000, i*5000).getInfo()

    records = [{'date': d['properties']['date'],'longitude':d['geometry']['coordinates'][0], 'latitude':d['geometry']['coordinates'][1], band:d['properties'][band]} for d in features_lst]
    return pd.DataFrame.from_records(records)


  df1 = make_dataset('QA')
  df1['Province'] = province
  df1['Town'] = town
  df1['QA'] = df1['QA'].apply(lambda x: format(x,'03b'))
  df1[['night/day', 'land/water']] = df1['QA'].str.extract(r'(.)(.*)')
  df1['night/day'] = df1['night/day'].astype(int)
  df1['land/water'] = df1['land/water'].apply(lambda x: int(x, 2))
  df1.drop(columns=['QA'], inplace=True)
  df2 = make_dataset('MaxFRP')

  df = df1.merge(df2, how="outer")
  df.fillna(0, inplace=True)
  df['fire'] = np.where(df['MaxFRP'] != 0, 1, 0)

  df.to_csv('dataset.csv')

  return df

## Example:

In [ ]:
coordinates = [[4.090573746751205, 36.583503499565424],
    [4.239575821946517, 36.583503499565424],
    [4.239575821946517, 36.66285977347944],
    [4.090573746751205, 36.66285977347944],
    [4.090573746751205, 36.583503499565424]]

start_date = '2021-08-08'
end_date = '2021-08-12'

province = 'Tizi Ouzou'
town = 'Larbaâ Nath Irathen'

df = wildfire_dataset(province=province, town=town, coordinates=coordinates, start_date=start_date, end_date=end_date)

The result will be a dataset with the following columns:

   date : the date of the sample

   longitude, latitude : the coordinates of the sample

   province, town : the province and town that you input

   night/day: 0 for night and 1 for day

   land/water: 0 water, 1 coast, 2 land, 3 missing data

   Maxfrp: Maximum fire radiative power
   
   fire : 0 non-fire, 1 fire